# Applicazione Web

- Per prima cosa predisponiamo tre cartelle:
    - __template__: conterrà i template delle nostre pagine.
    - __instance__: conterrà file di lavoro e temporanei dell'applicazione.
    - __static__: conterrà i file statici (es. css).

In [1]:
import os
if not os.path.isdir("templates"):
    os.mkdir("templates")

In [2]:
if not os.path.isdir("instance"):
    os.mkdir("instance")

In [3]:
if not os.path.isdir("static"):
    os.mkdir("static")

## Template

- Per prima scriviamo il template "astratto" per le nostre pagine.

In [137]:
%%writefile templates/master.html
<!DOCTYPE html>
<html>
  <head>
    <link rel="stylesheet" type="text/css" href="{{ url_for("static", filename="style.css") }}">
  </head>
  <body>
    <h1>{% block title %} {% endblock %}</h1>
    {% block main %}{% endblock %}
  </body>
</html>

Overwriting templates/master.html


- Passiamo alla scrittura dei template delle pagine vere e proprie.

In [167]:
%%writefile templates/home.html
{% extends "master.html" %}
{% block title %}Stock Exchange Market Predictor{% endblock %}
{% block main %}
<form method="GET" action="{{ url_for("predict") }}">
  <p>Open Price: <input type="number" step="0.01" name="Open">
  <p>Market Index: <select name="Index">
   <option value="1">NASDAQ</option>
   <option value="2">DJIA</option>
  </select></p>
  <p><button type="submit">Predict</button></p>
</form>
<img src="{{ url_for('static', filename='compare.png') }}" />
{% endblock %}

Overwriting templates/home.html


In [168]:
%%writefile templates/market_index.html
{% extends "master.html" %}
{% block title %}{{ index }}{% endblock %}
{% block main %}
<p>Open: {{ open_price }}</p>
<p>Predicted Direction: {{ direction }}</p> 
<p>Predicted Close: {{ close_price[0] | round(2) }}$</p>

<p> Classification ROI model: {{ roi | round(2) }}% with accuracy: {{ acc | round(2) }}%</p>
<p> RMSE Prediction model: {{ rmse | round(2) }}$</p>
<a href="{{ url_for("home") }}">Home</a>
{% if index == "NASDAQ" %}
    <img src="{{ url_for('static', filename='close-nasdaq-compare.png')}}" />
{% else %}
    <img src="{{ url_for('static', filename='close-djia-compare.png')}}" />
{% endif %}
{% endblock %}

Overwriting templates/market_index.html


## Routing

- Definiamo ora le funzioni che dovranno rispondere alle richieste.
- Predisponiamo le routine di risposta alle richieste inviate al sito.

In [169]:
%%writefile progetto.py
from flask import Flask, request, render_template
import pandas as pd
import pickle
import yfinance as yf

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("home.html")
    
@app.route("/predict")
def predict():
    open_price = float(request.args["Open"])
    
    n = 5

    if int(request.args["Index"]) == 1:
        df = nasdaq.tail(n)
        with open("instance/model_classification_nasdaq.bin", "rb") as f:
            model_classification = pickle.load(f)
        with open("instance/model_regression_nasdaq.bin", "rb") as f:
            model_regression = pickle.load(f)
        roi = model_stat.loc["nasdaq"]["roi"]
        rmse = model_stat.loc["nasdaq"]["rmse"]
        index = "NASDAQ"
        acc = model_stat.loc["nasdaq"]["acc"]
    else: 
        df = djia.tail(n)
        with open("instance/model_classification_djia.bin", "rb") as f:
            model_classification = pickle.load(f)
        with open("instance/model_regression_djia.bin", "rb") as f:
            model_regression = pickle.load(f)
        roi = model_stat.loc["djia"]["roi"]
        rmse = model_stat.loc["djia"]["rmse"]
        index = "DJIA"
        acc = model_stat.loc["djia"]["acc"]
    
    x = pd.DataFrame({"Open": [open_price]})
    
    x["VolumeLag1"] = df.iloc[n-1]["Volume"]
    
    for i in range(1,6):
        x["ROILag{}".format(i)] = df.iloc[n-i]["ROI"]
    for i in range(1,6):
        x["OpenLag{}".format(i)] = df.iloc[n-i]["Open"]
    for i in range(1,6):
        x["CloseLag{}".format(i)] = df.iloc[n-i]["Close"]
    for i in range(1,6):
        x["HighLag{}".format(i)] = df.iloc[n-i]["High"]
    for i in range(1,6):
        x["LowLag{}".format(i)] = df.iloc[n-i]["Low"]

    label = model_classification.predict(x.head(1))
    close = model_regression.predict(x.head(1))
    
    return render_template("market_index.html", \
                           index=index, \
                           open_price=open_price, \
                           direction=label, \
                           close_price=close, \
                           roi=roi, \
                           rmse=rmse, \
                           acc=acc)


def setup():
    model_stat = pd.read_csv (r'instance/model_stat.csv')
    model_stat.set_index('title', inplace=True)
    
    # Download file CSV dei due indici.
    nasdaq = yf.download("^IXIC", period="ytd")
    djia = yf.download("^DJI", period="ytd")
    
    # Drop del giorno odierno.
    nasdaq.drop(nasdaq.tail(1).index, inplace=True)
    djia.drop(djia.tail(1).index, inplace=True)
    
    nasdaq["ROI"] = 100*(nasdaq["Close"] - nasdaq["Open"]) / nasdaq["Open"] # Guadagno percentuale.
    djia["ROI"] = 100*(djia["Close"] - djia["Open"]) / djia["Open"] # Guadagno percentuale.
    
    return model_stat, nasdaq, djia
    
model_stat, nasdaq, djia = setup()

Overwriting progetto.py


## CSS

- File di stile per le nostre pagine.

In [1]:
%%writefile static/style.css

h1 {
    text-align: center;
}

div {
    width: 100%;
}

form {
    text-align: center;
}

img {
    width: 70%;
    margin-left: auto;
    margin-right: auto;
    display: block
}

div > * {
    text-align: center;
}

Overwriting static/style.css
